In [1]:
pd.options.display.max_rows = 9999
pd.options.display.max_columns = 9999

import sys
import missingno as msno
import matplotlib.pyplot as plt
from matplotlib import rc
import seaborn as sns
from PIL import Image
from functools import reduce

%matplotlib inline
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

In [2]:
df_credit_2020 = pd.read_excel("2020_credit_rating.xlsx")
df_credit_2020.head()

,Unnamed: 0,평가회사,평가대상회사,평가대상종류,공시일자,평가등급,등급감시,등급전망
0,1,한국기업평가,삼성엔지니어링(주),본평가,2020-01-02,A-,NaN,안정적
1,2,한국기업평가,삼화페인트공업(주),수시평가,2020-01-02,BBB+,NaN,안정적
2,3,나이스신용평가,예금보험공사,본평가,2020-01-08,AAA,NaN,안정적
3,4,한국기업평가,에스케이증권(주),본평가,2020-01-09,A,NaN,안정적
4,5,한국기업평가,우리종합금융(주),본평가 (정정),2020-01-09,A,NaN,안정적


In [3]:
credit_dlist={"D":0,"C-":1,"C":2,"C+":3,"CC-":4,"CC":5,"CC+":6,"CCC-":7,"CCC":8,"CCC+":9,
              "B-":10,"B":11,"B+":12,"BB-":13,"BB":14,"BB+":15,"BBB-":16,"BBB":17,"BBB+":18,
              "A-":19,"A":20,"A+":21,"AA-":22,"AA":23,"AA+":24,"AAA-":25,"AAA":26,"AAA+":27}


def screen_name(df):
    df = df.rename(columns = {"평가대상회사":"기업명"})
    df = df[["기업명","공시일자","평가등급"]]
    df["공시일자"] = df["공시일자"].apply(lambda x: int(str(x)[:4]))
    df["평가등급"][df["평가등급"] == "취소"] = np.nan
    df["기업명"] = df["기업명"].apply(lambda x: str(x).replace("(주)",""))
    df = df.dropna()
    df = df.drop_duplicates(['기업명'],keep="last").reset_index(drop=True)
    df['평가등급'] = df['평가등급'].apply(credit_dlist.get)
    return df

In [4]:
df_credit_2017 = screen_name(pd.read_excel("2017_credit_rating.xlsx"))
df_credit_2018 = screen_name(pd.read_excel("2018_credit_rating.xlsx"))
df_credit_2019 = screen_name(pd.read_excel("2019_credit_rating.xlsx"))
df_credit_2020 = screen_name(pd.read_excel("2020_credit_rating.xlsx"))

In [5]:
df_credit_2020.tail()

,기업명,공시일자,평가등급
206,에스케이네트웍스,2020,22
207,농협금융지주,2020,26
208,키움예스저축은행,2020,18
209,키움저축은행,2020,19
210,한전KPS,2020,23


In [6]:
df_2013_2016_credit = pd.read_excel("2013-2016_credit_rating.xlsx")
df_2013_2016_credit.columns = ["nnamed", "평가회사","기업명","평가","평가대상종류","공시일자","평가등급","등급감시","등급전망"]
df_2013_2016_credit["공시일자"] = df_2013_2016_credit["공시일자"].apply(lambda x: int(str(x)[:4]))

In [7]:
df_2013_2016_credit.head()

,nnamed,평가회사,기업명,평가,평가대상종류,공시일자,평가등급,등급감시,등급전망
0,2795,나이스신용평가,대우조선해양(주),기업신용평가,수시평가,2016,B+,NaN,부정적
1,2794,나이스신용평가,(주)엘에스네트웍스,기업신용평가,수시평가,2016,A-,NaN,부정적
2,2793,나이스신용평가,동국산업(주),기업신용평가,본평가,2016,BBB+,NaN,안정적
3,2792,한국신용평가,엘에스오토모티브(주),기업신용평가,본평가,2016,A-,NaN,안정적
4,2791,나이스신용평가,(주)두산캐피탈,기업신용평가,수시평가,2016,BBB-,↑,NaN


In [8]:
df_credit_2013 = screen_name(df_2013_2016_credit[df_2013_2016_credit["공시일자"] == 2013])
df_credit_2014 = screen_name(df_2013_2016_credit[df_2013_2016_credit["공시일자"] == 2014])
df_credit_2015 = screen_name(df_2013_2016_credit[df_2013_2016_credit["공시일자"] == 2015])
df_credit_2016 = screen_name(df_2013_2016_credit[df_2013_2016_credit["공시일자"] == 2016])

In [9]:
df_a_1 = pd.merge(df_credit_2014, df_credit_2015, on = "기업명",how = "inner").dropna()
df_a_1["Point"] = df_a_1["평가등급_x"] - df_a_1["평가등급_y"]
df_a_1["Point"][df_a_1["Point"].values != 0] =1
#df_a_1["Point"][df_a_1["Point"].values < 0] =-1
df_a_1 = df_a_1[["기업명","Point"]]
df_a_2 = pd.merge(df_a_1, df_credit_2015, on="기업명", how="inner")

In [10]:
df_b_1 = pd.merge(df_credit_2015, df_credit_2016, on = "기업명",how = "inner").dropna()
df_b_1["Point"] = df_b_1["평가등급_x"] - df_b_1["평가등급_y"]
df_b_1["Point"][df_b_1["Point"].values != 0] =1
#df_b_1["Point"][df_b_1["Point"].values < 0] =-1
df_b_1 = df_b_1[["기업명","Point"]]
df_b_2 = pd.merge(df_b_1, df_credit_2016, on="기업명", how="inner")

In [11]:
df_b_2

,기업명,Point,공시일자,평가등급
0,엠에스오토텍,0.0,2016,14.0
1,대한전선,0.0,2016,15.0
2,우룡,0.0,2016,13.0
3,우리종합금융,0.0,2016,19.0
4,중국농업은행서울지점,0.0,2016,26.0
5,한국철강,0.0,2016,20.0
6,에스케이건설,0.0,2016,19.0
7,오뚜기,0.0,2016,23.0
8,한국투자저축은행,0.0,2016,20.0
9,엘아이지,0.0,2016,21.0


In [12]:
df_c_1 = pd.merge(df_credit_2016, df_credit_2017, on = "기업명",how = "inner").dropna()
df_c_1["Point"] = df_c_1["평가등급_x"] - df_c_1["평가등급_y"]
df_c_1["Point"][df_c_1["Point"].values != 0] =1
#df_c_1["Point"][df_c_1["Point"].values < 0] =-1
df_c_1 = df_c_1[["기업명","Point"]]
df_c_2 = pd.merge(df_c_1, df_credit_2017, on="기업명", how="inner")

In [13]:
df_d_1 = pd.merge(df_credit_2017, df_credit_2018, on = "기업명",how = "inner").dropna()
df_d_1["Point"] = df_d_1["평가등급_x"] - df_d_1["평가등급_y"]
df_d_1["Point"][df_d_1["Point"].values != 0] =1
#df_d_1["Point"][df_d_1["Point"].values < 0] =-1
df_d_1 = df_d_1[["기업명","Point"]]
df_d_2 = pd.merge(df_d_1, df_credit_2018, on="기업명", how="inner")

In [14]:
df_e_1 = pd.merge(df_credit_2018, df_credit_2019, on = "기업명",how = "inner").dropna()
df_e_1["Point"] = df_e_1["평가등급_x"] - df_e_1["평가등급_y"]
df_e_1["Point"][df_e_1["Point"].values != 0] =1
#df_e_1["Point"][df_e_1["Point"].values > 0] =-1
df_e_1 = df_e_1[["기업명","Point"]]
df_e_2 = pd.merge(df_e_1, df_credit_2019, on="기업명", how="inner")

In [15]:
df_f_1 = pd.merge(df_credit_2019, df_credit_2020, on = "기업명",how = "inner").dropna()
df_f_1["Point"] = df_f_1["평가등급_x"] - df_f_1["평가등급_y"]
df_f_1["Point"][df_f_1["Point"].values != 0] =1
#df_f_1["Point"][df_f_1["Point"].values < 0] =-1
df_f_1 = df_f_1[["기업명","Point"]]
df_f_2 = pd.merge(df_f_1, df_credit_2020, on="기업명", how="inner")

In [16]:
df_credit_2015 = df_a_2
df_credit_2016 = df_b_2
df_credit_2017 = df_c_2
df_credit_2018 = df_d_2
df_credit_2019 = df_e_2
df_credit_2020 = df_f_2

In [17]:
df_esg = pd.read_csv("2011-2019-ESG.tsv", sep='\t')
df_esg.head()

,NO,기업명,기업코드,ESG등급,지배구조,사회,환경,평가년도,비고
0,6567,AJ네트웍스,95570,B,B,B,C이하,2019,NaN
1,6566,AJ렌터카,68400,C이하,C,B,B,2019,NaN
2,6565,AK홀딩스,6840,B,B,B,C이하,2019,NaN
3,6564,AP시스템,265520,C이하,B,C이하,C이하,2019,NaN
4,6563,BGF,27410,B,B,A,C이하,2019,NaN


In [18]:
df_esg = df_esg.drop(['NO', '비고'], axis=1).dropna()
df_esg['ESG등급'] = df_esg['ESG등급'].apply({"S":70.0,"A+":60.0,"A":50.0,"B+":40.0,"B":30.0,"B이하":30.0,"C":20.0,"C이하":20.0,"D":10.0}.get) 
df_esg['지배구조'] = df_esg['지배구조'].apply({"S":70.0,"A+":60.0,"A":50.0,"B+":40.0,"B":30.0,"B이하":30.0,"C":20.0,"C이하":20.0,"D":10.0}.get) 
df_esg['사회']    = df_esg['사회'].apply({"S":70.0,"A+":60.0,"A":50.0,"B+":40.0,"B":30.0,"B이하":30.0,"C":20.0,"C이하":20.0,"D":10.0}.get) 
df_esg['환경']    = df_esg['환경'].apply({"S":70.0,"A+":60.0,"A":50.0,"B+":40.0,"B":30.0,"B이하":30.0,"C":20.0,"C이하":20.0,"D":10.0}.get) 
df_esg["기업코드"] = df_esg["기업코드"].apply(lambda x: str(x))
dflist = list(df_esg["기업코드"].values)
for i in range(len(df_esg)):
    if len(dflist[i]) < 3:
        dflist[i] = "0000" + dflist[i]
    elif len(dflist[i]) < 4:
        dflist[i] ="000" + dflist[i]
    elif len(dflist[i]) < 5:
        dflist[i] ="00" + dflist[i]
    elif len(dflist[i]) < 6:
        dflist[i] = "0" + dflist[i]
df_esg["기업코드"] = dflist 

In [19]:
df_esg.tail()

,기업명,기업코드,ESG등급,지배구조,사회,환경,평가년도
6555,화천기계,010660,30.0,30.0,30.0,30.0,2011
6559,효성,004800,30.0,40.0,30.0,40.0,2011
6561,후성,093370,30.0,20.0,30.0,30.0,2011
6562,휴니드테크놀러지스,005870,40.0,20.0,40.0,30.0,2011
6564,휴켐스,069260,30.0,40.0,30.0,40.0,2011


In [20]:
mod = sys.modules[__name__]

for i in range(2011,2020):
    setattr(mod, 'df_esg_{}'.format(i),df_esg[df_esg["평가년도"]==i])

In [21]:
df_esg_2019.tail()

,기업명,기업코드,ESG등급,지배구조,사회,환경,평가년도
918,휴온스글로벌,084110,30.0,40.0,40.0,20.0,2019
919,휴젤,145020,20.0,30.0,20.0,20.0,2019
920,휴켐스,069260,30.0,30.0,40.0,30.0,2019
922,흥국화재,000540,30.0,30.0,30.0,20.0,2019
923,흥아해운,003280,20.0,30.0,20.0,20.0,2019


In [22]:
df2 = pd.read_csv("marcap-2011.csv")
df2.head()

,Code,Name,Close,Changes,ChagesRatio,Volume,Amount,Open,High,Low,Marcap,MarcapRatio,Stocks,ForeignShares,ForeignRatio,Rank,Date
0,005930,삼성전자,958000,9000,1.0,265562,254246714088,955000,966000,950000,141112764846000,11.27,147299337,74518914.0,50.59,1,2011-01-03
1,005490,POSCO,486000,-1000,-0.2,170542,82998761000,487000,489500,482000,42372801810000,3.38,87186835,43377735.0,49.75,2,2011-01-03
2,005380,현대차,177000,3500,2.0,556135,97468816500,174000,177000,173500,38988936783000,3.11,220276479,94245489.0,42.79,3,2011-01-03
3,009540,현대중공업,435000,-8000,-1.8,256635,111611214500,443000,443500,431000,33060000000000,2.64,76000000,15350207.0,20.20,4,2011-01-03
4,012330,현대모비스,294500,10000,3.5,212902,61730122000,287500,294500,285000,28667767653500,2.29,97343863,46134411.0,47.39,5,2011-01-03


In [23]:
# file_list_a = ["marcap-2011.csv","marcap-2012.csv","marcap-2013.csv","marcap-2014.csv","marcap-2015.csv",
#                "marcap-2016.csv","marcap-2017.csv","marcap-2018.csv","marcap-2019.csv"]

# def screen_data(df):
#     df = df[["Code","Name","Date","Marcap"]]
#     df["Date"] = df["Date"].apply(lambda x: int(str(x).replace("-","")))
#     df = df.drop_duplicates(['Name'],keep="last")
#     df = df.rename(columns = {"Code":"기업코드"})
#     return df
    
# for i in range(2011,2020):
#     setattr(mod, 'df_market_{}'.format(i),screen_data(pd.read_csv(file_list_a[i-2011])))

In [24]:
# df_market_2019.tail()

In [25]:
df3 = pd.read_excel("영업활동현금흐름.xlsx")
df3.tail()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
2212,950130,엑세스바이오,12,천원,-422239.0,-5730652.0,-5445311.0,2553507.0,2967795.0,-2633921.0,1587126.0,1845660.0,-2351654.0,1915059.0,NaN,NaN
2213,950140,잉글우드랩,12,천원,12115959.0,-9368274.0,-8980775.0,-3506442.0,-471765.0,6338815.0,NaN,NaN,NaN,NaN,NaN,NaN
2214,950160,코오롱티슈진,12,천원,-43856498.0,-36048325.0,-38815829.0,-3498024.0,-5610900.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2215,950170,JTC,02,천원,NaN,13138388.0,3706136.0,46183777.0,9076766.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2216,950180,SNK,07,천원,32821083.0,22198273.0,25146954.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
df_f = ["영업활동현금흐름.xlsx","EBITDA.xlsx","CAPEX.xlsx","당기순이익.xlsx","자산총계.xlsx","매출액.xlsx","EBIT.xlsx",
       "부채비율.xlsx","차입부채_EBITDA.xlsx","차입금의존도.xlsx","총차입부채.xlsx"]

def change_columns(df):
    columns = ["종목코드","기업명","결산월","단위","2019","2018","2017","2016","2015","2014","2013","2012","2011","2010","2009","2008"]
    df.columns = columns
    df = df.rename(columns = {"종목코드":"기업코드"})
    return df

for i in range(1,12):
    setattr(mod, 'df_balance_{}'.format(i), change_columns(pd.read_excel(df_f[i-1])[9:]))
    

In [27]:
df_balance_3[df_balance_3.values == "N/A(IFRS)"] = np.nan
df_balance_10[df_balance_10.values == "완전잠식"] = np.nan
for i in range(4,16):
    df_balance_10.iloc[:,i] = df_balance_10.iloc[:,i].apply(lambda x: float(x))
    df_balance_8.iloc[:,i] = df_balance_10.iloc[:,i].apply(lambda x: float(x))

In [28]:
def merge_code(dfa,dfb):
    dfb = dfb[["기업코드","기업명"]][dfb["기업명"].isin(dfa["기업명"])]
    dfa = pd.merge(dfa,dfb, on="기업명",how="inner")
    return dfa

In [29]:
df_credit_2020.tail()

,기업명,Point,공시일자,평가등급
191,유안타증권,0,2020,21
192,동국제강,0,2020,16
193,한국해외인프라도시개발지원공사,0,2020,26
194,중국은행서울지점,0,2020,26
195,동국산업,0,2020,18


In [30]:
list_credit = [df_credit_2013,df_credit_2014,df_credit_2015,df_credit_2016,df_credit_2017,
             df_credit_2018,df_credit_2019]

list_esg = [df_esg_2011,df_esg_2012,df_esg_2013,df_esg_2014,df_esg_2015,df_esg_2016,
            df_esg_2017,df_esg_2018,df_esg_2019]

# list_marcap = [df_market_2011,df_market_2012,df_market_2013,df_market_2014,df_market_2015,df_market_2016,
#                df_market_2017,df_market_2018,df_market_2019]

list_balance = [df_balance_1,df_balance_2,df_balance_3,df_balance_4,df_balance_5,
               df_balance_6,df_balance_7,df_balance_8,df_balance_9,df_balance_10,df_balance_11]

In [31]:
for i in range(2013,2020):
    setattr(mod, 'df_rating_{}'.format(i), merge_code(list_credit[i-2013],list_esg[i-2013]))

for k in range(2011, 2020):
    for i in range(1,12):
        setattr(mod, 'df_{}_{}'.format(k,i), pd.DataFrame(list_balance[i-1].iloc[:,[0,2023-k]]))

In [32]:
df_columns = ["기업코드","영업활동현금흐름","EBITDA","CAPEX","당기순이익","자산총계","매출액","EBIT","부채비율"]
       #"차입부채_EBITDA","총차입부채","차입금의존도"]

In [33]:
data_frames_2011 = [df_2011_1,df_2011_2,df_2011_3,df_2011_4,df_2011_5,df_2011_6,df_2011_7,df_2011_8,]#df_2011_9,df_2011_10,df_2011_11,]
df_merged_2011 = reduce(lambda left,right: pd.merge(left,right,on=['기업코드'], how='inner'), data_frames_2011).dropna()
df_merged_2011.columns = df_columns

data_frames_2012 = [df_2012_1,df_2012_2,df_2012_3,df_2012_4,df_2012_5,df_2012_6,df_2012_7,df_2012_8,]#df_2012_9,df_2012_10,df_2012_11,]
df_merged_2012 = reduce(lambda left,right: pd.merge(left,right,on=['기업코드'], how='inner'), data_frames_2012).dropna()
df_merged_2012.columns = df_columns

data_frames_2013 = [df_2013_1,df_2013_2,df_2013_3,df_2013_4,df_2013_5,df_2013_6,df_2013_7,df_2013_8,]#df_2013_9,df_2013_10,df_2013_11,]
df_merged_2013 = reduce(lambda left,right: pd.merge(left,right,on=['기업코드'], how='inner'), data_frames_2013).dropna()
df_merged_2013.columns = df_columns

data_frames_2014 = [df_2014_1,df_2014_2,df_2014_3,df_2014_4,df_2014_5,df_2014_6,df_2014_7,df_2014_8,]#df_2014_9,df_2014_10,df_2014_11,]
df_merged_2014 = reduce(lambda left,right: pd.merge(left,right,on=['기업코드'], how='inner'), data_frames_2014).dropna()
df_merged_2014.columns = df_columns

data_frames_2015 = [df_2015_1,df_2015_2,df_2015_3,df_2015_4,df_2015_5,df_2015_6,df_2015_7,df_2015_8,]#df_2015_9,df_2015_10,df_2015_11,]
df_merged_2015 = reduce(lambda left,right: pd.merge(left,right,on=['기업코드'], how='inner'), data_frames_2015).dropna()
df_merged_2015.columns = df_columns

data_frames_2016 = [df_2016_1,df_2016_2,df_2016_3,df_2016_4,df_2016_5,df_2016_6,df_2016_7,df_2016_8,]#df_2016_9,df_2016_10,df_2016_11,]
df_merged_2016 = reduce(lambda left,right: pd.merge(left,right,on=['기업코드'], how='inner'), data_frames_2016).dropna()
df_merged_2016.columns = df_columns

data_frames_2017 = [df_2017_1,df_2017_2,df_2017_3,df_2017_4,df_2017_5,df_2017_6,df_2017_7,df_2017_8,]#df_2017_9,df_2017_10,df_2017_11,]
df_merged_2017 = reduce(lambda left,right: pd.merge(left,right,on=['기업코드'], how='inner'), data_frames_2017).dropna()
df_merged_2017.columns = df_columns

data_frames_2018 = [df_2018_1,df_2018_2,df_2018_3,df_2018_4,df_2018_5,df_2018_6,df_2018_7,df_2018_8,]#df_2018_9,df_2018_10,df_2018_11,]
df_merged_2018 = reduce(lambda left,right: pd.merge(left,right,on=['기업코드'], how='inner'), data_frames_2018).dropna()
df_merged_2018.columns = df_columns

data_frames_2019 = [df_2019_1,df_2019_2,df_2019_3,df_2019_4,df_2019_5,df_2019_6,df_2019_7,df_2019_8,]#df_2019_9,df_2019_10,df_2019_11,]
df_merged_2019 = reduce(lambda left,right: pd.merge(left,right,on=['기업코드'], how='inner'), data_frames_2019).dropna()
df_merged_2019.columns = df_columns

In [34]:
df_merged_2019.describe()

,영업활동현금흐름,EBITDA,CAPEX,당기순이익,자산총계,매출액,EBIT,부채비율
count,2.187000e+03,2.187000e+03,2.187000e+03,2.187000e+03,2.187000e+03,2.187000e+03,2.187000e+03,2187.000000
mean,8.991773e+07,1.233352e+08,1.020767e+08,2.626147e+07,1.532547e+09,1.074795e+09,5.411809e+07,23.385167
std,1.057824e+09,1.347377e+09,1.107775e+09,4.942229e+08,1.158036e+10,7.030940e+09,6.334466e+08,22.393188
min,-2.947197e+09,-2.996260e+08,-8.999175e+08,-2.872078e+09,3.385050e+05,0.000000e+00,-1.359382e+09,0.000000
25%,-5.337045e+05,1.170422e+06,1.009242e+06,-4.336090e+06,7.177917e+07,4.100846e+07,-1.261744e+06,6.635000
50%,6.315040e+06,8.406333e+06,5.582711e+06,2.268541e+06,1.553955e+08,1.172023e+08,3.782730e+06,21.090000
75%,2.715564e+07,3.403316e+07,2.226423e+07,1.319283e+07,4.287833e+08,3.551279e+08,1.929850e+07,35.775000
max,4.538292e+10,5.736615e+10,3.981829e+10,2.173886e+10,3.525645e+11,2.304009e+11,2.776851e+10,589.310000


In [35]:
df_merged_2011["FCF"] = df_merged_2011["영업활동현금흐름"] - df_merged_2011["CAPEX"]
#df_merged_2011["EBIT_매출액"] = df_merged_2011["EBIT"]/df_merged_2011["매출액"]

df_merged_2012["FCF"] = df_merged_2012["영업활동현금흐름"] - df_merged_2012["CAPEX"]
#df_merged_2012["EBIT_매출액"] = df_merged_2012["EBIT"]/df_merged_2012["매출액"]

df_merged_2013["FCF"] = df_merged_2013["영업활동현금흐름"] - df_merged_2013["CAPEX"]
#df_merged_2013["EBIT_매출액"] = df_merged_2013["EBIT"]/df_merged_2013["매출액"]

df_merged_2014["FCF"] = df_merged_2014["영업활동현금흐름"] - df_merged_2014["CAPEX"]
#df_merged_2014["EBIT_매출액"] = df_merged_2011["EBIT"]/df_merged_2011["매출액"]

df_merged_2015["FCF"] = df_merged_2015["영업활동현금흐름"] - df_merged_2015["CAPEX"]
#df_merged_2015["EBIT_매출액"] = df_merged_2015["EBIT"]/df_merged_2015["매출액"]

df_merged_2016["FCF"] = df_merged_2016["영업활동현금흐름"] - df_merged_2016["CAPEX"]
#df_merged_2016["EBIT_매출액"] = df_merged_2016["EBIT"]/df_merged_2016["매출액"]

df_merged_2017["FCF"] = df_merged_2017["영업활동현금흐름"] - df_merged_2017["CAPEX"]
#df_merged_2017["EBIT_매출액"] = df_merged_2017["EBIT"]/df_merged_2017["매출액"]

df_merged_2018["FCF"] = df_merged_2018["영업활동현금흐름"] - df_merged_2018["CAPEX"]
#df_merged_2018["EBIT_매출액"] = df_merged_2018["EBIT"]/df_merged_2018["매출액"]

df_merged_2019["FCF"] = df_merged_2019["영업활동현금흐름"] - df_merged_2019["CAPEX"]
#df_merged_2019["EBIT_매출액"] = df_merged_2019["EBIT"]/df_merged_2019["매출액"]

In [36]:
df_merged_2013.describe()

,영업활동현금흐름,EBITDA,당기순이익,자산총계,매출액,EBIT,부채비율
count,1.930000e+03,1.930000e+03,1.930000e+03,1.930000e+03,1.930000e+03,1.930000e+03,1930.000000
mean,7.815510e+07,1.024315e+08,3.545669e+07,1.162102e+09,1.005666e+09,5.752737e+07,24.822342
std,1.124417e+09,1.302359e+09,7.460273e+08,8.162859e+09,6.915794e+09,8.805874e+08,22.345318
min,-1.230924e+09,-9.557412e+08,-1.909683e+09,5.464350e+05,0.000000e+00,-1.028047e+09,0.000000
25%,1.985782e+05,2.091437e+06,-6.874500e+05,4.730186e+07,3.425458e+07,4.709878e+05,7.482500
50%,5.606137e+06,7.604753e+06,2.843341e+06,1.117685e+08,9.883951e+07,4.286388e+06,23.030000
75%,1.949150e+07,2.427409e+07,1.074248e+07,3.049699e+08,2.970550e+08,1.608148e+07,38.957500
max,4.670744e+10,5.323043e+10,3.047476e+10,2.140750e+11,2.286927e+11,3.678501e+10,508.280000


In [37]:
data_frames_a_2011 = [df_merged_2011,df_esg_2011]
df_merged_b_2011 = reduce(lambda left,right: pd.merge(left,right,on=['기업코드'], how='inner'), data_frames_a_2011).dropna()

data_frames_a_2012 = [df_merged_2012,df_esg_2012]
df_merged_b_2012 = reduce(lambda left,right: pd.merge(left,right,on=['기업코드'], how='inner'), data_frames_a_2012).dropna()

data_frames_a_2013 = [df_merged_2013,df_esg_2013]
df_merged_b_2013 = reduce(lambda left,right: pd.merge(left,right,on=['기업코드'], how='inner'), data_frames_a_2013).dropna()

data_frames_a_2014 = [df_merged_2014,df_esg_2014]
df_merged_b_2014 = reduce(lambda left,right: pd.merge(left,right,on=['기업코드'], how='inner'), data_frames_a_2014).dropna()

data_frames_a_2015 = [df_merged_2015,df_esg_2015]
df_merged_b_2015 = reduce(lambda left,right: pd.merge(left,right,on=['기업코드'], how='inner'), data_frames_a_2015).dropna()

data_frames_a_2016 = [df_merged_2016,df_esg_2016]
df_merged_b_2016 = reduce(lambda left,right: pd.merge(left,right,on=['기업코드'], how='inner'), data_frames_a_2016).dropna()

data_frames_a_2017 = [df_merged_2017,df_esg_2017]
df_merged_b_2017 = reduce(lambda left,right: pd.merge(left,right,on=['기업코드'], how='inner'), data_frames_a_2017).dropna()

data_frames_a_2018 = [df_merged_2018,df_esg_2018]
df_merged_b_2018 = reduce(lambda left,right: pd.merge(left,right,on=['기업코드'], how='inner'), data_frames_a_2018).dropna()

data_frames_a_2019 = [df_merged_2019,df_esg_2019]
df_merged_b_2019 = reduce(lambda left,right: pd.merge(left,right,on=['기업코드'], how='inner'), data_frames_a_2019).dropna()

In [38]:
df_merged_b_2011.columns

Index(['기업코드', '영업활동현금흐름', 'EBITDA', 'CAPEX', '당기순이익',
       '자산총계', '매출액', 'EBIT', '부채비율', 'FCF', '기업명', 'ESG등급',
       '지배구조', '사회', '환경', '평가년도'],
      dtype='object')

In [39]:
df_merged_b_2011.head()

,기업코드,영업활동현금흐름,EBITDA,CAPEX,당기순이익,자산총계,매출액,EBIT,부채비율,FCF,기업명,ESG등급,지배구조,사회,환경,평가년도
0,000020,46244032.0,34559742.0,42917488,17679223.0,3.394315e+08,2.345620e+08,22102734.0,8.40,3.32654e+06,동화약품,30.0,40.0,30.0,30.0,2011
1,000040,-8429641.0,6582686.0,1033477,1904749.0,1.333378e+08,1.277202e+08,2493135.0,7.73,-9.46312e+06,KR모터스,30.0,30.0,30.0,30.0,2011
2,000070,159327065.0,88142799.0,-216567063,168087785.0,1.884481e+09,1.180816e+09,43630799.0,18.23,3.75894e+08,삼양홀딩스,30.0,30.0,30.0,40.0,2011
3,000100,72377881.0,70508222.0,15357641,94217471.0,1.314066e+09,6.792310e+08,52731142.0,0.69,5.70202e+07,유한양행,30.0,30.0,30.0,30.0,2011
4,000120,111199423.0,199353088.0,-238398595,80994345.0,3.595163e+09,2.587848e+09,125019927.0,23.27,3.49598e+08,CJ대한통운,40.0,30.0,40.0,30.0,2011


In [40]:
df_merged_b_2011 = df_merged_b_2011.iloc[:,[0,2,4,5,6,8,9,11,12,13,14,]]
df_merged_b_2012 = df_merged_b_2012.iloc[:,[0,2,4,5,6,8,9,11,12,13,14,]]
df_merged_b_2013 = df_merged_b_2013.iloc[:,[0,2,4,5,6,8,9,11,12,13,14,]]
df_merged_b_2014 = df_merged_b_2014.iloc[:,[0,2,4,5,6,8,9,11,12,13,14,]]
df_merged_b_2015 = df_merged_b_2015.iloc[:,[0,2,4,5,6,8,9,11,12,13,14,]]
df_merged_b_2016 = df_merged_b_2016.iloc[:,[0,2,4,5,6,8,9,11,12,13,14,]]
df_merged_b_2017 = df_merged_b_2017.iloc[:,[0,2,4,5,6,8,9,11,12,13,14,]]
df_merged_b_2018 = df_merged_b_2018.iloc[:,[0,2,4,5,6,8,9,11,12,13,14,]]
df_merged_b_2019 = df_merged_b_2019.iloc[:,[0,2,4,5,6,8,9,11,12,13,14,]]

In [41]:
df_merged_b_2011.head()

,기업코드,EBITDA,당기순이익,자산총계,매출액,부채비율,FCF,ESG등급,지배구조,사회,환경
0,000020,34559742.0,17679223.0,3.394315e+08,2.345620e+08,8.40,3.32654e+06,30.0,40.0,30.0,30.0
1,000040,6582686.0,1904749.0,1.333378e+08,1.277202e+08,7.73,-9.46312e+06,30.0,30.0,30.0,30.0
2,000070,88142799.0,168087785.0,1.884481e+09,1.180816e+09,18.23,3.75894e+08,30.0,30.0,30.0,40.0
3,000100,70508222.0,94217471.0,1.314066e+09,6.792310e+08,0.69,5.70202e+07,30.0,30.0,30.0,30.0
4,000120,199353088.0,80994345.0,3.595163e+09,2.587848e+09,23.27,3.49598e+08,40.0,30.0,40.0,30.0


In [42]:
df_merged_z_2012 = df_merged_b_2012[df_merged_b_2012["기업코드"].isin(list(set(df_merged_b_2011["기업코드"])&set(df_merged_b_2012["기업코드"])))].reset_index(drop=True)
df_merged_z_2011 = df_merged_b_2011[df_merged_b_2011["기업코드"].isin(list(set(df_merged_b_2011["기업코드"])&set(df_merged_b_2012["기업코드"])))].reset_index(drop=True)

In [43]:
print(len(df_merged_z_2012))
print(len(df_merged_z_2011))

259
259


In [44]:
k = pd.DataFrame(df_merged_z_2012.groupby("기업코드").size())
k[k[0]==2]

,0
기업코드,


In [45]:
# df_merged_z_2011[df_merged_z_2011['기업코드'] == "086280"] #7,5,25

In [46]:
# df_market_2012[df_market_2012['기업코드'] == "004840"]

In [47]:
df_merged_b_2012.columns

Index(['기업코드', 'EBITDA', '당기순이익', '자산총계', '매출액',
       '부채비율', 'FCF', 'ESG등급', '지배구조', '사회', '환경'],
      dtype='object')

In [48]:
columns = ['기업코드', '당기순이익', '자산총계', '매출액','부채비율', 'FCF', 'ESG등급', '지배구조', '사회', '환경']

In [49]:
df_merged_c_2012 = pd.DataFrame(columns=columns)
df_merged_c_2012["기업코드"] = df_merged_z_2012["기업코드"]
#df_merged_c_2012['EBITDA'] = (df_merged_z_2012['EBITDA'] - df_merged_z_2011['EBITDA']) / np.abs(df_merged_z_2011['EBITDA'])
df_merged_c_2012['매출액'] = (df_merged_z_2012['매출액'] - df_merged_z_2011['매출액']) / np.abs(df_merged_z_2011['매출액'])
df_merged_c_2012['당기순이익'] = (df_merged_z_2012['당기순이익'] - df_merged_z_2011['당기순이익'] )/ np.abs(df_merged_z_2011['당기순이익'])
df_merged_c_2012["FCF"] = (df_merged_z_2012["FCF"] - df_merged_z_2011["FCF"]) / np.abs(df_merged_z_2011["FCF"])
#df_merged_c_2012['총차입부채'] = (df_merged_z_2012['총차입부채'] - df_merged_z_2011['총차입부채']) / np.abs(df_merged_z_2011['총차입부채'])
df_merged_c_2012['자산총계'] = (df_merged_z_2012['자산총계'] - df_merged_z_2011['자산총계']) / np.abs(df_merged_z_2011['자산총계'])
#df_merged_c_2012['EBIT_매출액'] = (df_merged_z_2012['EBIT_매출액'] - df_merged_z_2011['EBIT_매출액']) / np.abs(df_merged_z_2011['EBIT_매출액'])
df_merged_c_2012['부채비율'] = (df_merged_z_2012['부채비율'] - df_merged_z_2011['부채비율']) / np.abs(df_merged_z_2011['부채비율'])
#df_merged_c_2012['차입금의존도'] = (df_merged_z_2012['차입금의존도'] - df_merged_z_2011['차입금의존도']) / np.abs(df_merged_z_2011['차입금의존도'])
#df_merged_c_2012['Marcap'] = (df_merged_z_2012['Marcap'] - df_merged_z_2011['Marcap']) / np.abs(df_merged_z_2011['Marcap'])
df_merged_c_2012['ESG등급'] = (df_merged_z_2012['ESG등급'] - df_merged_z_2011['ESG등급'])
df_merged_c_2012['지배구조'] = (df_merged_z_2012['지배구조'] - df_merged_z_2011['지배구조'])
df_merged_c_2012['사회'] = (df_merged_z_2012['사회'] - df_merged_z_2011['사회'])
df_merged_c_2012['환경'] = (df_merged_z_2012['환경'] - df_merged_z_2011['환경'])

In [50]:
df_merged_c_2012.head()

,기업코드,당기순이익,자산총계,매출액,부채비율,FCF,ESG등급,지배구조,사회,환경
0,000020,-0.925832,-0.044624,-0.047706,-0.633333,4.55105,0.0,0.0,0.0,0.0
1,000040,-5.321856,-0.105075,-0.233272,0.128072,0.417188,0.0,0.0,0.0,0.0
2,000070,-0.228113,0.458438,0.578625,0.289632,-1.93141,0.0,0.0,0.0,0.0
3,000100,-0.142667,0.067743,0.143142,-0.159420,0.147976,10.0,10.0,0.0,10.0
4,000120,-0.141027,0.020677,0.015309,0.006016,-1.39915,-10.0,0.0,0.0,0.0


In [51]:
df_total_2013 = pd.merge(df_rating_2013,df_merged_c_2012.dropna(), on="기업코드", how="inner")

In [52]:
df_total_2013.describe()

,공시일자,평가등급,당기순이익,자산총계,매출액,부채비율,ESG등급,지배구조,사회,환경
count,19.0,18.000000,19.000000,19.000000,19.000000,19.000000,19.00000,19.000000,19.000000,19.000000
mean,2013.0,17.444444,-3.278296,0.037447,0.009986,29.741776,0.00000,-2.631579,0.000000,2.631579
std,0.0,4.475058,9.937440,0.146991,0.164184,111.090742,7.45356,6.533763,6.666667,5.619515
min,2013.0,8.000000,-37.118480,-0.172277,-0.342598,-0.478303,-10.00000,-20.000000,-10.000000,-10.000000
25%,2013.0,17.000000,-1.455483,-0.037948,-0.102774,0.001479,0.00000,-5.000000,0.000000,0.000000
50%,2013.0,18.500000,-0.651966,0.006851,0.041436,0.087491,0.00000,0.000000,0.000000,0.000000
75%,2013.0,20.000000,0.142428,0.096647,0.106648,0.180700,0.00000,0.000000,0.000000,10.000000
max,2013.0,24.000000,11.112157,0.524061,0.245059,482.000000,20.00000,10.000000,20.000000,10.000000


In [53]:
len(df_total_2013)

19

In [54]:
df_merged_z_2013 = df_merged_b_2013[df_merged_b_2013["기업코드"].isin(list(set(df_merged_b_2013["기업코드"])&set(df_merged_b_2012["기업코드"])))].reset_index(drop=True)
df_merged_x_2012 = df_merged_b_2012[df_merged_b_2012["기업코드"].isin(list(set(df_merged_b_2013["기업코드"])&set(df_merged_b_2012["기업코드"])))].reset_index(drop=True)

In [55]:
print(len(df_merged_z_2013))
print(len(df_merged_x_2012))

572
572


In [56]:
k = pd.DataFrame(df_merged_z_2013.groupby("기업코드").size())
k[k[0]==2]

,0
기업코드,


In [57]:
df_merged_z_2013[df_merged_z_2013['기업코드'] == "035420"] #7,5,25

,기업코드,EBITDA,당기순이익,자산총계,매출액,부채비율,FCF,ESG등급,지배구조,사회,환경
452,035420,638235669.0,1.895289e+09,2.697747e+09,2.259139e+09,13.04,2.57839e+08,40.0,40.0,40.0,30.0


In [58]:
#df_merged_z_2013.groupby("기업코드").size()

In [59]:
df_merged_c_2013 = pd.DataFrame(columns=columns)
df_merged_c_2013["기업코드"] = df_merged_z_2013["기업코드"]
df_merged_c_2013['매출액'] = (df_merged_z_2013['매출액'] - df_merged_x_2012['매출액']) / np.abs(df_merged_x_2012['매출액'])
#df_merged_c_2013['EBITDA'] = (df_merged_z_2013['EBITDA'] - df_merged_z_2012['EBITDA']) / np.abs(df_merged_z_2012['EBITDA'])
df_merged_c_2013['당기순이익'] = (df_merged_z_2013['당기순이익'] - df_merged_x_2012['당기순이익'] )/ np.abs(df_merged_x_2012['당기순이익'])
df_merged_c_2013["FCF"] = (df_merged_z_2013["FCF"] - df_merged_x_2012["FCF"]) / np.abs(df_merged_x_2012["FCF"])
#df_merged_c_2013['총차입부채'] = (df_merged_z_2013['총차입부채'] - df_merged_x_2012['총차입부채']) / np.abs(df_merged_x_2012['총차입부채'])
df_merged_c_2013['자산총계'] = (df_merged_z_2013['자산총계'] - df_merged_x_2012['자산총계']) / np.abs(df_merged_x_2012['자산총계'])
#df_merged_c_2013['EBIT_매출액'] = (df_merged_z_2013['EBIT_매출액'] - df_merged_x_2012['EBIT_매출액']) / np.abs(df_merged_x_2012['EBIT_매출액'])
df_merged_c_2013['부채비율'] = (df_merged_z_2013['부채비율'] - df_merged_x_2012['부채비율']) / np.abs(df_merged_x_2012['부채비율'])
#df_merged_c_2013['차입금의존도'] = (df_merged_z_2013['차입금의존도'] - df_merged_x_2012['차입금의존도']) / np.abs(df_merged_x_2012['차입금의존도'])
#df_merged_c_2013['Marcap'] = (df_merged_z_2013['Marcap'] - df_merged_x_2012['Marcap']) / np.abs(df_merged_x_2012['Marcap'])
df_merged_c_2013['ESG등급'] = (df_merged_z_2013['ESG등급'] - df_merged_x_2012['ESG등급'])
df_merged_c_2013['지배구조'] = (df_merged_z_2013['지배구조'] - df_merged_x_2012['지배구조'])
df_merged_c_2013['사회'] = (df_merged_z_2013['사회'] - df_merged_x_2012['사회'])
df_merged_c_2013['환경'] = (df_merged_z_2013['환경'] - df_merged_x_2012['환경'])

In [60]:
df_total_2014 = pd.merge(df_rating_2014,df_merged_c_2013.dropna(), on="기업코드", how="inner")

In [61]:
len(df_total_2014)

124

In [62]:
df_total_2014.describe()

,공시일자,평가등급,당기순이익,자산총계,매출액,부채비율,ESG등급,지배구조,사회,환경
count,124.0,124.000000,124.000000,124.000000,124.000000,124.000000,124.000000,124.000000,124.000000,124.000000
mean,2014.0,19.669355,-2.378545,0.022450,-0.000543,inf,0.161290,-0.887097,0.403226,0.000000
std,0.0,3.583435,11.521243,0.127331,0.162461,NaN,5.098042,6.248525,6.426469,4.597631
min,2014.0,8.000000,-101.365014,-0.735048,-0.816426,-0.615819,-10.000000,-20.000000,-20.000000,-10.000000
25%,2014.0,18.000000,-1.026211,-0.022999,-0.067459,-0.079795,0.000000,0.000000,0.000000,0.000000
50%,2014.0,20.000000,-0.161609,0.011413,0.016141,-0.002737,0.000000,0.000000,0.000000,0.000000
75%,2014.0,22.000000,0.223941,0.073109,0.068309,0.066849,0.000000,0.000000,0.000000,0.000000
max,2014.0,26.000000,6.821596,0.524059,0.557066,inf,20.000000,10.000000,20.000000,20.000000


In [63]:
df_merged_z_2014 = df_merged_b_2014[df_merged_b_2014["기업코드"].isin(list(set(df_merged_b_2014["기업코드"])&set(df_merged_b_2013["기업코드"])))].reset_index(drop=True)
df_merged_x_2013 = df_merged_b_2013[df_merged_b_2013["기업코드"].isin(list(set(df_merged_b_2014["기업코드"])&set(df_merged_b_2013["기업코드"])))].reset_index(drop=True)

In [64]:
print(len(df_merged_z_2014))
print(len(df_merged_x_2013))

600
600


In [65]:
k = pd.DataFrame(df_merged_x_2013.groupby("기업코드").size())
k[k[0]==2]

,0
기업코드,


In [66]:
df_merged_c_2014 = pd.DataFrame(columns=columns)
df_merged_c_2014["기업코드"] = df_merged_z_2014["기업코드"]
df_merged_c_2014['매출액'] = (df_merged_z_2014['매출액'] - df_merged_x_2013['매출액']) / np.abs(df_merged_x_2013['매출액'])
df_merged_c_2014['EBITDA'] = (df_merged_z_2014['EBITDA'] - df_merged_z_2013['EBITDA']) / np.abs(df_merged_z_2013['EBITDA'])
df_merged_c_2014['당기순이익'] = (df_merged_z_2014['당기순이익'] - df_merged_x_2013['당기순이익'] )/ np.abs(df_merged_x_2013['당기순이익'])
df_merged_c_2014["FCF"] = (df_merged_z_2014["FCF"] - df_merged_x_2013["FCF"]) / np.abs(df_merged_x_2013["FCF"])
#df_merged_c_2014['총차입부채'] = (df_merged_z_2014['총차입부채'] - df_merged_x_2013['총차입부채']) / np.abs(df_merged_x_2013['총차입부채'])
df_merged_c_2014['자산총계'] = (df_merged_z_2014['자산총계'] - df_merged_x_2013['자산총계']) / np.abs(df_merged_x_2013['자산총계'])
#df_merged_c_2014['EBIT_매출액'] = (df_merged_z_2014['EBIT_매출액'] - df_merged_x_2013['EBIT_매출액']) / np.abs(df_merged_x_2013['EBIT_매출액'])
df_merged_c_2014['부채비율'] = (df_merged_z_2014['부채비율'] - df_merged_x_2013['부채비율']) / np.abs(df_merged_x_2013['부채비율'])
#df_merged_c_2014['차입금의존도'] = (df_merged_z_2014['차입금의존도'] - df_merged_x_2013['차입금의존도']) / np.abs(df_merged_x_2013['차입금의존도'])
#df_merged_c_2014['Marcap'] = (df_merged_z_2014['Marcap'] - df_merged_x_2013['Marcap']) / np.abs(df_merged_x_2013['Marcap'])
df_merged_c_2014['ESG등급'] = (df_merged_z_2014['ESG등급'] - df_merged_x_2013['ESG등급'])
df_merged_c_2014['지배구조'] = (df_merged_z_2014['지배구조'] - df_merged_x_2013['지배구조'])
df_merged_c_2014['사회'] = (df_merged_z_2014['사회'] - df_merged_x_2013['사회'])
df_merged_c_2014['환경'] = (df_merged_z_2014['환경'] - df_merged_x_2013['환경'])

In [67]:
df_total_2015 = pd.merge(df_rating_2015,df_merged_c_2014.dropna(), on="기업코드", how="inner")

In [68]:
len(df_total_2015)

100

In [69]:
df_total_2015.describe()

,Point,공시일자,평가등급,당기순이익,자산총계,매출액,부채비율,ESG등급,지배구조,사회,환경,EBITDA
count,100.000000,100.0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,0.260000,2015.0,19.620000,-0.286868,0.019614,0.024713,0.007500,-0.100000,0.300000,-0.300000,0.100000,35.606954
std,0.440844,0.0,3.765285,6.012413,0.144772,0.130266,0.295470,5.024184,7.581876,5.403889,5.411361,151.308836
min,0.000000,2015.0,0.000000,-41.633650,-0.669216,-0.204232,-0.965827,-20.000000,-30.000000,-10.000000,-20.000000,-82.965830
25%,0.000000,2015.0,18.000000,-0.680787,-0.024137,-0.035766,-0.091380,0.000000,0.000000,0.000000,0.000000,-0.851950
50%,0.000000,2015.0,20.000000,-0.042155,0.027009,0.012875,-0.020069,0.000000,0.000000,0.000000,0.000000,0.998105
75%,1.000000,2015.0,22.000000,0.824017,0.086832,0.062499,0.052719,0.000000,0.000000,0.000000,0.000000,13.173934
max,1.000000,2015.0,26.000000,26.173609,0.331980,0.894145,1.809459,10.000000,30.000000,20.000000,10.000000,1091.810557


In [70]:
df_merged_z_2015 = df_merged_b_2015[df_merged_b_2015["기업코드"].isin(list(set(df_merged_b_2014["기업코드"])&set(df_merged_b_2015["기업코드"])))].reset_index(drop=True)
df_merged_x_2014 = df_merged_b_2014[df_merged_b_2014["기업코드"].isin(list(set(df_merged_b_2014["기업코드"])&set(df_merged_b_2015["기업코드"])))].reset_index(drop=True)

In [71]:
print(len(df_merged_z_2015))
print(len(df_merged_x_2014))

614
614


In [72]:
k = pd.DataFrame(df_merged_z_2015.groupby("기업코드").size())
k[k[0]==2]

,0
기업코드,


In [73]:
df_merged_z_2015[df_merged_z_2015['기업코드'] == "102260"] #211,222,314,432,454 / 55,260,272,281,323,377,379,467

,기업코드,EBITDA,당기순이익,자산총계,매출액,부채비율,FCF,ESG등급,지배구조,사회,환경
576,102260,88143335.0,43556795.0,801695229.0,871224899.0,30.23,2.61585e+07,30.0,20.0,30.0,30.0


In [74]:
df_merged_c_2015 = pd.DataFrame(columns = columns)
df_merged_c_2015["기업코드"] = df_merged_z_2015["기업코드"]
df_merged_c_2015['매출액'] = (df_merged_z_2015['매출액'] - df_merged_x_2014['매출액']) / np.abs(df_merged_x_2014['매출액'])
#df_merged_c_2015['EBITDA'] = (df_merged_z_2015['EBITDA'] - df_merged_z_2014['EBITDA']) / np.abs(df_merged_z_2014['EBITDA'])
df_merged_c_2015['당기순이익'] = (df_merged_z_2015['당기순이익'] - df_merged_x_2014['당기순이익'] )/ np.abs(df_merged_x_2014['당기순이익'])
df_merged_c_2015["FCF"] = (df_merged_z_2015["FCF"] - df_merged_x_2014["FCF"]) / np.abs(df_merged_x_2014["FCF"])
#df_merged_c_2015['총차입부채'] = (df_merged_z_2015['총차입부채'] - df_merged_x_2014['총차입부채']) / np.abs(df_merged_x_2014['총차입부채'])
df_merged_c_2015['자산총계'] = (df_merged_z_2015['자산총계'] - df_merged_x_2014['자산총계']) / np.abs(df_merged_x_2014['자산총계'])
#df_merged_c_2015['EBIT_매출액'] = (df_merged_z_2015['EBIT_매출액'] - df_merged_x_2014['EBIT_매출액']) / np.abs(df_merged_x_2014['EBIT_매출액'])
df_merged_c_2015['부채비율'] = (df_merged_z_2015['부채비율'] - df_merged_x_2014['부채비율']) / np.abs(df_merged_x_2014['부채비율'])
#df_merged_c_2015['차입금의존도'] = (df_merged_z_2015['차입금의존도'] - df_merged_x_2014['차입금의존도']) / np.abs(df_merged_x_2014['차입금의존도'])
#df_merged_c_2015['Marcap'] = (df_merged_z_2015['Marcap'] - df_merged_x_2014['Marcap']) / np.abs(df_merged_x_2014['Marcap'])
df_merged_c_2015['ESG등급'] = (df_merged_z_2015['ESG등급'] / df_merged_x_2014['ESG등급'])
df_merged_c_2015['지배구조'] = (df_merged_z_2015['지배구조'] / df_merged_x_2014['지배구조'])
df_merged_c_2015['사회'] = (df_merged_z_2015['사회'] - df_merged_x_2014['사회'])
df_merged_c_2015['환경'] = (df_merged_z_2015['환경'] - df_merged_x_2014['환경'])

In [75]:
df_total_2016 = pd.merge(df_rating_2016,df_merged_c_2015.dropna(), on="기업코드", how="inner")

In [76]:
len(df_total_2016)

90

In [77]:
df_total_2016.describe()

,Point,공시일자,평가등급,당기순이익,자산총계,매출액,부채비율,ESG등급,지배구조,사회,환경
count,90.000000,90.0,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000
mean,0.200000,2016.0,19.666667,0.689560,0.056196,-0.009062,0.008354,0.952407,0.900556,-1.222222,0.222222
std,0.402241,0.0,3.365389,16.228487,0.153046,0.146716,0.285883,0.119731,0.319258,5.963895,4.735185
min,0.000000,2016.0,8.000000,-52.152999,-0.424681,-0.301252,-1.000000,0.600000,0.250000,-20.000000,-10.000000
25%,0.000000,2016.0,18.000000,-0.445983,-0.026642,-0.092680,-0.074616,1.000000,0.666667,0.000000,0.000000
50%,0.000000,2016.0,20.000000,0.085803,0.036670,0.004401,0.014507,1.000000,1.000000,0.000000,0.000000
75%,0.000000,2016.0,22.000000,1.318133,0.109449,0.044815,0.092522,1.000000,1.000000,0.000000,0.000000
max,1.000000,2016.0,26.000000,130.438887,0.667023,0.730684,1.157629,1.333333,2.000000,10.000000,10.000000


In [78]:
df_merged_z_2016 = df_merged_b_2016[df_merged_b_2016["기업코드"].isin(list(set(df_merged_b_2015["기업코드"])&set(df_merged_b_2016["기업코드"])))].reset_index(drop=True)
df_merged_x_2015 = df_merged_b_2015[df_merged_b_2015["기업코드"].isin(list(set(df_merged_b_2015["기업코드"])&set(df_merged_b_2016["기업코드"])))].reset_index(drop=True)

In [79]:
print(len(df_merged_z_2016))
print(len(df_merged_x_2015))

624
624


In [80]:
k = pd.DataFrame(df_merged_x_2015.groupby("기업코드").size())
k[k[0]==2]

,0
기업코드,


In [81]:
df_merged_x_2015[df_merged_x_2015['기업코드'] == "102260"] #14,32,137,189,234,435,486 / 62,296,303,313,316,358,393,419,421,512

,기업코드,EBITDA,당기순이익,자산총계,매출액,부채비율,FCF,ESG등급,지배구조,사회,환경
580,102260,88143335.0,43556795.0,801695229.0,871224899.0,30.23,2.61585e+07,30.0,20.0,30.0,30.0


In [82]:
df_merged_c_2016 = pd.DataFrame(columns=columns)
df_merged_c_2016["기업코드"] = df_merged_z_2016["기업코드"]
df_merged_c_2016['매출액'] = (df_merged_z_2016['매출액'] - df_merged_x_2015['매출액']) / np.abs(df_merged_x_2015['매출액'])
#df_merged_c_2016['EBITDA'] = (df_merged_z_2016['EBITDA'] - df_merged_z_2015['EBITDA']) / np.abs(df_merged_z_2015['EBITDA'])
df_merged_c_2016['당기순이익'] = (df_merged_z_2016['당기순이익'] - df_merged_x_2015['당기순이익'] )/ np.abs(df_merged_x_2015['당기순이익'])
df_merged_c_2016["FCF"] = (df_merged_z_2016["FCF"] - df_merged_x_2015["FCF"]) / np.abs(df_merged_x_2015["FCF"])
#df_merged_c_2016['총차입부채'] = (df_merged_z_2016['총차입부채'] - df_merged_x_2015['총차입부채']) / np.abs(df_merged_x_2015['총차입부채'])
df_merged_c_2016['자산총계'] = (df_merged_z_2016['자산총계'] - df_merged_x_2015['자산총계']) / np.abs(df_merged_x_2015['자산총계'])
#df_merged_c_2016['EBIT_매출액'] = (df_merged_z_2016['EBIT_매출액'])/ np.abs(df_merged_x_2015['EBIT_매출액'])
df_merged_c_2016['부채비율'] = (df_merged_z_2016['부채비율'] - df_merged_x_2015['부채비율']) / np.abs(df_merged_x_2015['부채비율'])
#df_merged_c_2016['차입금의존도'] = (df_merged_z_2016['차입금의존도'] - df_merged_x_2015['차입금의존도']) / np.abs(df_merged_x_2015['차입금의존도'])
#df_merged_c_2016['Marcap'] = (df_merged_z_2016['Marcap'] - df_merged_x_2015['Marcap']) / np.abs(df_merged_x_2015['Marcap'])
df_merged_c_2016['ESG등급'] = (df_merged_z_2016['ESG등급'] - df_merged_x_2015['ESG등급'])
df_merged_c_2016['지배구조'] = (df_merged_z_2016['지배구조'] - df_merged_x_2015['지배구조'])
df_merged_c_2016['사회'] = (df_merged_z_2016['사회'] - df_merged_x_2015['사회'])
df_merged_c_2016['환경'] = (df_merged_z_2016['환경'] - df_merged_x_2015['환경'])

In [83]:
df_total_2017 = pd.merge(df_rating_2017,df_merged_c_2016.dropna(), on="기업코드", how="inner")

In [84]:
len(df_total_2017)

99

In [85]:
df_total_2017.describe()

,Point,공시일자,평가등급,당기순이익,자산총계,매출액,부채비율,ESG등급,지배구조,사회,환경
count,99.000000,99.0,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000
mean,0.202020,2017.0,19.666667,0.737697,0.037322,0.026012,inf,2.323232,1.212121,4.343434,1.212121
std,0.403551,0.0,3.667285,6.246714,0.162004,0.184768,NaN,5.501073,10.229282,9.275997,4.796412
min,0.000000,2017.0,8.000000,-17.995944,-0.430549,-0.518533,-1.000000,-10.000000,-20.000000,-10.000000,-10.000000
25%,0.000000,2017.0,18.000000,-0.314853,-0.017196,-0.039899,-0.114372,0.000000,-5.000000,0.000000,0.000000
50%,0.000000,2017.0,20.000000,0.102346,0.028080,0.017002,-0.046353,0.000000,0.000000,0.000000,0.000000
75%,0.000000,2017.0,22.000000,0.717493,0.067282,0.082475,0.041290,0.000000,10.000000,10.000000,0.000000
max,1.000000,2017.0,26.000000,54.388561,1.175192,1.105910,inf,30.000000,50.000000,30.000000,10.000000


In [86]:
df_merged_z_2017 = df_merged_b_2017[df_merged_b_2017["기업코드"].isin(list(set(df_merged_b_2016["기업코드"])&set(df_merged_b_2017["기업코드"])))].reset_index(drop=True)
df_merged_x_2016 = df_merged_b_2016[df_merged_b_2016["기업코드"].isin(list(set(df_merged_b_2016["기업코드"])&set(df_merged_b_2017["기업코드"])))].reset_index(drop=True)

In [87]:
print(len(df_merged_z_2017))
print(len(df_merged_x_2016))

643
643


In [88]:
k = pd.DataFrame(df_merged_z_2017.groupby("기업코드").size())
k[k[0]==2]

,0
기업코드,


In [89]:
df_merged_z_2017[df_merged_z_2017['기업코드'] == "069640"] #14, 32, 48, 136, 190, 238, 449, 503, 570,392,393 / 36, 106, 301, 303, 308, 391, 439, 449,477

,기업코드,EBITDA,당기순이익,자산총계,매출액,부채비율,FCF,ESG등급,지배구조,사회,환경
533,069640,14923388.0,7471384.0,212873830.0,328898003.0,6.11,-694616,30.0,30.0,30.0,30.0


In [90]:
df_merged_c_2017 = pd.DataFrame(columns=columns)
df_merged_c_2017["기업코드"] = df_merged_z_2017["기업코드"]
#df_merged_c_2017['EBITDA'] = (df_merged_z_2017['EBITDA'] - df_merged_z_2016['EBITDA']) / np.abs(df_merged_z_2016['EBITDA'])
df_merged_c_2017['매출액'] = (df_merged_z_2017['매출액'] - df_merged_x_2016['매출액']) / np.abs(df_merged_x_2016['매출액'])
df_merged_c_2017['당기순이익'] = (df_merged_z_2017['당기순이익'] - df_merged_x_2016['당기순이익'] )/ np.abs(df_merged_x_2016['당기순이익'])
df_merged_c_2017["FCF"] = (df_merged_z_2017["FCF"] - df_merged_x_2016["FCF"]) / np.abs(df_merged_x_2016["FCF"])
#df_merged_c_2017['총차입부채'] = (df_merged_z_2017['총차입부채'] - df_merged_x_2016['총차입부채']) / np.abs(df_merged_x_2016['총차입부채'])
df_merged_c_2017['자산총계'] = (df_merged_z_2017['자산총계'] - df_merged_x_2016['자산총계']) / np.abs(df_merged_x_2016['자산총계'])
#df_merged_c_2017['EBIT_매출액'] = (df_merged_z_2017['EBIT_매출액'] - df_merged_x_2016['EBIT_매출액']) / np.abs(df_merged_x_2016['EBIT_매출액'])
df_merged_c_2017['부채비율'] = (df_merged_z_2017['부채비율'] - df_merged_x_2016['부채비율']) / np.abs(df_merged_x_2016['부채비율'])
#df_merged_c_2017['차입금의존도'] = (df_merged_z_2017['차입금의존도'] - df_merged_x_2016['차입금의존도']) / np.abs(df_merged_x_2016['차입금의존도'])
#df_merged_c_2017['Marcap'] = (df_merged_z_2017['Marcap'] - df_merged_x_2016['Marcap']) / np.abs(df_merged_x_2016['Marcap'])
df_merged_c_2017['ESG등급'] = (df_merged_z_2017['ESG등급'] - df_merged_x_2016['ESG등급'])
df_merged_c_2017['지배구조'] = (df_merged_z_2017['지배구조'] - df_merged_x_2016['지배구조'])
df_merged_c_2017['사회'] = (df_merged_z_2017['사회'] - df_merged_x_2016['사회'])
df_merged_c_2017['환경'] = (df_merged_z_2017['환경'] - df_merged_x_2016['환경'])

In [91]:
df_total_2018 = pd.merge(df_rating_2018,df_merged_c_2017, on="기업코드", how="inner")

In [92]:
len(df_total_2018)

99

In [93]:
df_total_2018.describe()

,Point,공시일자,평가등급,당기순이익,자산총계,매출액,부채비율,ESG등급,지배구조,사회,환경
count,99.000000,99.0,99.000000,99.000000,99.000000,99.000000,97.000000,99.000000,99.000000,99.000000,99.000000
mean,0.111111,2018.0,20.000000,0.906949,0.026963,0.080057,0.025960,0.000000,3.636364,-0.303030,-0.404040
std,0.315869,0.0,3.380617,7.358640,0.138850,0.227637,0.365322,5.345225,8.625371,8.742874,3.757765
min,0.000000,2018.0,10.000000,-13.377627,-0.378768,-0.508561,-0.460117,-10.000000,-20.000000,-30.000000,-10.000000
25%,0.000000,2018.0,18.000000,-0.557309,-0.021203,0.002876,-0.114004,0.000000,0.000000,0.000000,0.000000
50%,0.000000,2018.0,20.000000,0.001372,0.023250,0.043943,-0.045917,0.000000,0.000000,0.000000,0.000000
75%,0.000000,2018.0,22.000000,1.055537,0.064810,0.137753,0.079249,0.000000,10.000000,0.000000,0.000000
max,1.000000,2018.0,26.000000,66.982370,0.545686,1.644660,2.264550,10.000000,30.000000,20.000000,10.000000


In [94]:
df_merged_z_2018 = df_merged_b_2018[df_merged_b_2018["기업코드"].isin(list(set(df_merged_b_2017["기업코드"])&set(df_merged_b_2018["기업코드"])))].reset_index(drop=True)
df_merged_x_2017 = df_merged_b_2017[df_merged_b_2017["기업코드"].isin(list(set(df_merged_b_2017["기업코드"])&set(df_merged_b_2018["기업코드"])))].reset_index(drop=True)

In [95]:
print(len(df_merged_z_2018))
print(len(df_merged_x_2017))

669
669


In [96]:
k = pd.DataFrame(df_merged_x_2017.groupby("기업코드").size())
k[k[0]==2]

,0
기업코드,


In [97]:
df_merged_x_2017[df_merged_x_2017['기업코드'] == "227840"]# 212,315,319,346,452,474,509,518,230,231 / 36,111,307,309,314,401,449,462,488,594,596

,기업코드,EBITDA,당기순이익,자산총계,매출액,부채비율,FCF,ESG등급,지배구조,사회,환경
662,227840,8883214.0,12026649.0,198540566.0,145238609.0,0.0,7.55999e+06,30.0,30.0,30.0,30.0


In [98]:
df_merged_c_2018 = pd.DataFrame(columns=columns)
df_merged_c_2018["기업코드"] = df_merged_z_2018["기업코드"]
df_merged_c_2018['매출액'] = (df_merged_z_2018['매출액'] - df_merged_x_2017['매출액']) / np.abs(df_merged_x_2017['매출액'])
#df_merged_c_2018['EBITDA'] = (df_merged_z_2018['EBITDA'] - df_merged_z_2017['EBITDA']) / np.abs(df_merged_z_2017['EBITDA'])
df_merged_c_2018['당기순이익'] = (df_merged_z_2018['당기순이익'] - df_merged_x_2017['당기순이익'] )/ np.abs(df_merged_x_2017['당기순이익'])
df_merged_c_2018["FCF"] = (df_merged_z_2018["FCF"] - df_merged_x_2017["FCF"]) / np.abs(df_merged_x_2017["FCF"])
#df_merged_c_2018['총차입부채'] = (df_merged_z_2018['총차입부채'] - df_merged_x_2017['총차입부채']) / np.abs(df_merged_x_2017['총차입부채'])
df_merged_c_2018['자산총계'] = (df_merged_z_2018['자산총계'] - df_merged_x_2017['자산총계']) / np.abs(df_merged_x_2017['자산총계'])
#df_merged_c_2018['EBIT_매출액'] = (df_merged_z_2018['EBIT_매출액'] - df_merged_x_2017['EBIT_매출액']) / np.abs(df_merged_x_2017['EBIT_매출액'])
df_merged_c_2018['부채비율'] = (df_merged_z_2018['부채비율'] - df_merged_x_2017['부채비율']) / np.abs(df_merged_x_2017['부채비율'])
#df_merged_c_2018['차입금의존도'] = (df_merged_z_2018['차입금의존도'] - df_merged_x_2017['차입금의존도']) / np.abs(df_merged_x_2017['차입금의존도'])
#df_merged_c_2018['Marcap'] = (df_merged_z_2018['Marcap'] - df_merged_x_2017['Marcap']) / np.abs(df_merged_x_2017['Marcap'])
df_merged_c_2018['ESG등급'] = (df_merged_z_2018['ESG등급'] - df_merged_x_2017['ESG등급'])
df_merged_c_2018['지배구조'] = (df_merged_z_2018['지배구조'] - df_merged_x_2017['지배구조'])
df_merged_c_2018['사회'] = (df_merged_z_2018['사회'] - df_merged_x_2017['사회'])
df_merged_c_2018['환경'] = (df_merged_z_2018['환경'] - df_merged_x_2017['환경'])

In [99]:
df_total_2019 = pd.merge(df_rating_2019,df_merged_c_2018, on="기업코드", how="inner")

In [100]:
len(df_total_2019)

78

In [101]:
df_total_2019.describe()

,Point,공시일자,평가등급,당기순이익,자산총계,매출액,부채비율,ESG등급,지배구조,사회,환경
count,78.000000,78.0,78.000000,78.000000,78.000000,78.000000,76.000000,78.000000,78.000000,78.000000,78.000000
mean,0.205128,2019.0,20.102564,-1.450256,0.048724,0.040259,-0.049370,1.025641,0.256410,1.153846,0.000000
std,0.406409,0.0,3.496156,8.038991,0.269313,0.118501,0.218461,6.155927,9.799862,7.892256,5.345225
min,0.000000,2019.0,10.000000,-64.831159,-0.142596,-0.259187,-0.949320,-10.000000,-20.000000,-20.000000,-10.000000
25%,0.000000,2019.0,18.250000,-0.692388,-0.020357,-0.005577,-0.129090,0.000000,-10.000000,0.000000,0.000000
50%,0.000000,2019.0,21.000000,-0.242192,0.012040,0.030202,-0.009487,0.000000,0.000000,0.000000,0.000000
75%,0.000000,2019.0,22.750000,0.312378,0.047127,0.073695,0.047130,0.000000,10.000000,10.000000,0.000000
max,1.000000,2019.0,26.000000,5.187521,2.270395,0.652676,0.709318,20.000000,30.000000,20.000000,10.000000


In [102]:
df_merged_z_2019 = df_merged_b_2019[df_merged_b_2019["기업코드"].isin(list(set(df_merged_b_2018["기업코드"])&set(df_merged_b_2019["기업코드"])))].reset_index(drop=True)
df_merged_x_2018 = df_merged_b_2018[df_merged_b_2018["기업코드"].isin(list(set(df_merged_b_2018["기업코드"])&set(df_merged_b_2019["기업코드"])))].reset_index(drop=True)

In [103]:
print(len(df_merged_z_2019))
print(len(df_merged_x_2018))

667
667


In [104]:
k = pd.DataFrame(df_merged_z_2019.groupby("기업코드").size())
k[k[0]==2]

,0
기업코드,


In [105]:
df_merged_z_2019[df_merged_z_2019['기업코드'] == "181710"] #205,305,309,337,368,440,462,497,505,553,222,223 / 11,50,55,91,370,446,462,559,565

,기업코드,EBITDA,당기순이익,자산총계,매출액,부채비율,FCF,ESG등급,지배구조,사회,환경
643,181710,156747726.0,23173213.0,2.500321e+09,1.488597e+09,5.61,129309272.0,30.0,40.0,30.0,20.0


In [106]:
df_merged_c_2019 = pd.DataFrame(columns=columns)
df_merged_c_2019["기업코드"] = df_merged_z_2019["기업코드"]
df_merged_c_2019['매출액'] = (df_merged_z_2019['매출액'] - df_merged_x_2018['매출액']) / np.abs(df_merged_x_2018['매출액'])
#df_merged_c_2019['EBITDA'] = (df_merged_z_2019['EBITDA'] - df_merged_z_2018['EBITDA']) / np.abs(df_merged_z_2018['EBITDA'])
df_merged_c_2019['당기순이익'] = (df_merged_z_2019['당기순이익'] - df_merged_x_2018['당기순이익'] )/ np.abs(df_merged_x_2018['당기순이익'])
df_merged_c_2019["FCF"] = (df_merged_z_2019["FCF"] - df_merged_x_2018["FCF"]) / np.abs(df_merged_x_2018["FCF"])
#df_merged_c_2019['총차입부채'] = (df_merged_z_2019['총차입부채'] - df_merged_x_2018['총차입부채']) / np.abs(df_merged_x_2018['총차입부채'])
df_merged_c_2019['자산총계'] = (df_merged_z_2019['자산총계'] - df_merged_x_2018['자산총계']) / np.abs(df_merged_x_2018['자산총계'])
#df_merged_c_2019['EBIT_매출액'] = (df_merged_z_2019['EBIT_매출액'] - df_merged_x_2018['EBIT_매출액']) / np.abs(df_merged_x_2018['EBIT_매출액'])
df_merged_c_2019['부채비율'] = (df_merged_z_2019['부채비율'] - df_merged_x_2018['부채비율']) / np.abs(df_merged_x_2018['부채비율'])
#df_merged_c_2019['차입금의존도'] = (df_merged_z_2019['차입금의존도'] - df_merged_x_2018['차입금의존도']) / np.abs(df_merged_x_2018['차입금의존도'])
#df_merged_c_2019['Marcap'] = (df_merged_z_2019['Marcap'] - df_merged_x_2018['Marcap']) / np.abs(df_merged_x_2018['Marcap'])
df_merged_c_2019['ESG등급'] = (df_merged_z_2019['ESG등급'] - df_merged_x_2018['ESG등급'])
df_merged_c_2019['지배구조'] = (df_merged_z_2019['지배구조'] - df_merged_x_2018['지배구조'])
df_merged_c_2019['사회'] = (df_merged_z_2019['사회'] - df_merged_x_2018['사회'])
df_merged_c_2019['환경'] = (df_merged_z_2019['환경'] - df_merged_x_2018['환경'])

In [107]:
df_rating_2020 = merge_code(df_credit_2020, df_esg_2019)

In [108]:
df_total_2020 = pd.merge(df_rating_2020,df_merged_c_2019, on="기업코드", how="inner")

In [109]:
len(df_total_2020)

36

In [110]:
df_total_2020.describe()

,Point,공시일자,평가등급,당기순이익,자산총계,매출액,부채비율,ESG등급,지배구조,사회,환경
count,36.000000,36.0,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000
mean,0.138889,2020.0,20.472222,-0.885145,0.046263,0.089469,inf,-1.111111,2.777778,0.833333,-4.722222
std,0.350736,0.0,3.299230,3.650601,0.115126,0.445742,NaN,5.745944,6.594851,7.699722,6.540472
min,0.000000,2020.0,10.000000,-15.206419,-0.110393,-0.185682,-0.729461,-10.000000,-10.000000,-10.000000,-20.000000
25%,0.000000,2020.0,19.000000,-0.759476,-0.020388,-0.035770,-0.000631,0.000000,0.000000,-2.500000,-10.000000
50%,0.000000,2020.0,21.500000,-0.061785,0.022994,0.012593,0.044265,0.000000,0.000000,0.000000,-5.000000
75%,0.000000,2020.0,22.250000,0.496426,0.077081,0.040487,0.181824,0.000000,10.000000,10.000000,0.000000
max,1.000000,2020.0,26.000000,3.862332,0.531109,2.626014,inf,10.000000,20.000000,10.000000,10.000000


In [111]:
df_sum_2015_2020 = pd.concat([df_total_2015,df_total_2016,df_total_2017,df_total_2018,df_total_2019,df_total_2020]).replace(np.inf,np.nan).dropna()

In [112]:
# df_sum_2015_2020.to_pickle("df.pkl")

In [113]:
df_x = df_sum_2015_2020[["매출액","당기순이익","FCF","자산총계","부채비율","지배구조"]]#,"ESG등급", "사회","환경"

In [114]:
df_y = df_sum_2015_2020[["Point"]]

In [115]:
len(df_y[df_y ==1].dropna())

26

In [116]:
len(df_x)

100

In [117]:
len(df_x.values[df_x.values>30])

1

In [118]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_x, df_y,test_size=0.3)

In [119]:
from sklearn.preprocessing import RobustScaler
robust_scaler = RobustScaler()
rb = robust_scaler.fit(X_train)
X_train = robust_scaler.transform(X_train)
X_test = robust_scaler.transform(X_test)

In [120]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [121]:
y_pred = list(model.predict(X_test))
y_true = list(np.array(y_test))

In [122]:
print(model.predict(X_test).shape)
print(y_test.shape)

(30,)
(30, 1)


In [123]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
precision_score(y_true, y_pred, average='weighted')

0.8306818181818181

In [124]:
print(len(y_test[y_test==-1].dropna()))
print(len(y_test[y_test==1].dropna()))

0
3


In [125]:
confusion_matrix(y_true, y_pred, labels=[1,0])

array([[ 1,  2],
       [ 7, 20]])

In [126]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [127]:
y_pred = list(model.predict(X_test))
y_true = list(np.array(y_test))

In [128]:
from sklearn.metrics import precision_score
precision_score(y_true, y_pred, average='weighted')

0.792

In [129]:
df_sum_2015_2020.head()

,기업명,Point,공시일자,평가등급,기업코드,당기순이익,자산총계,매출액,부채비율,FCF,ESG등급,지배구조,사회,환경,EBITDA
0,두산인프라코어,0.0,2015,19.0,042670,1.237466,0.041452,-0.006239,0.035191,3.40264,0.0,0.0,0.0,0.0,79.101483
1,화승알앤에이,1.0,2015,16.0,013520,1.272470,0.024419,0.039400,0.010800,1.75531,0.0,0.0,0.0,0.0,4.172955
2,화신,0.0,2015,19.0,010690,26.173609,0.186656,-0.114551,0.183771,0.217625,0.0,10.0,0.0,0.0,-0.865303
3,현대글로비스,0.0,2015,23.0,086280,0.114193,0.222321,0.082478,-0.026074,-0.55238,-10.0,0.0,-10.0,0.0,0.023819
4,금호타이어,1.0,2015,19.0,073240,0.303596,0.014941,-0.070480,-0.080963,-0.129578,10.0,10.0,10.0,10.0,70.180632


In [130]:
df_sum_2015_2020.columns = ["X1","X2","X3","X4","X5","X6","X7","X8","X9","X10","X11","X12","X13","X14"]

ValueError: Length mismatch: Expected axis has 15 elements, new values have 14 elements

In [ ]:
df_x = df_sum_2015_2020.iloc[:,5:]

In [ ]:
df_y = df_sum_2015_2020.iloc[:,1]

In [ ]:
from sklearn.preprocessing import RobustScaler
robust_scaler = RobustScaler()
robust_scaler.fit(df_x)
output = robust_scaler.transform(df_x)
output = pd.DataFrame(output, columns=df_x.columns, index=list(df_x.index.values))
output["Y"] = df_y
output.tail()

In [ ]:
df_sum_2015_2020.columns

In [ ]:
from sklearn.linear_model import LinearRegression
model = sm.OLS.from_formula("Y ~X6+X7+X8+X9+X10+X11+X12+X13+X14+0", data=output)
result1 = model.fit()
print(result1.summary())

In [ ]:
from sklearn.linear_model import LogisticRegression
model = sm.Logit.from_formula("Y ~X6+X7+X8+X9+X10+X11+X12+X13+X14+0", data=output)
result1 = model.fit()
print(result1.summary())

In [ ]:
df_merged_q_2015 = df_merged_b_2014[df_merged_b_2015["기업코드"].isin(list(set(df_merged_b_2015["기업코드"])&set(df_merged_b_2014["기업코드"])&set(df_merged_b_2013["기업코드"])&set(df_merged_b_2012["기업코드"])))].reset_index(drop=True)
df_merged_q_2014 = df_merged_b_2013[df_merged_b_2014["기업코드"].isin(list(set(df_merged_b_2015["기업코드"])&set(df_merged_b_2014["기업코드"])&set(df_merged_b_2013["기업코드"])&set(df_merged_b_2012["기업코드"])))].reset_index(drop=True)
df_merged_q_2013 = df_merged_b_2012[df_merged_b_2013["기업코드"].isin(list(set(df_merged_b_2015["기업코드"])&set(df_merged_b_2014["기업코드"])&set(df_merged_b_2013["기업코드"])&set(df_merged_b_2012["기업코드"])))].reset_index(drop=True)
df_merged_q_2012 = df_merged_b_2012[df_merged_b_2012["기업코드"].isin(list(set(df_merged_b_2015["기업코드"])&set(df_merged_b_2014["기업코드"])&set(df_merged_b_2013["기업코드"])&set(df_merged_b_2012["기업코드"])))].reset_index(drop=True)

In [ ]:
df_merged_q_2012.head()

In [ ]:
len(df_merged_q_2013)

In [ ]:
df_merged_t_2014 = pd.DataFrame(columns=columns)
df_merged_t_2014["기업코드"] = df_merged_q_2014["기업코드"]
df_merged_t_2014['매출액'] = ((df_merged_q_2014['매출액'] + df_merged_q_2013['매출액']/2) - (df_merged_q_2013['매출액'] + df_merged_q_2012['매출액']/2))\
                                / np.abs(df_merged_q_2013['매출액']+df_merged_q_2012['매출액']/2)
df_merged_t_2014['EBITDA'] = ((df_merged_q_2014['EBITDA']+df_merged_q_2013['EBITDA'])/2-(df_merged_q_2013['EBITDA']+df_merged_q_2012['EBITDA'])/2 )\
                                / np.abs((df_merged_q_2013['EBITDA']+df_merged_q_2012['EBITDA'])/2)
df_merged_t_2014['당기순이익'] = (df_merged_z_2014['당기순이익'] - df_merged_x_2013['당기순이익'] )/ np.abs(df_merged_x_2013['당기순이익'])
df_merged_t_2014["FCF"] = (df_merged_z_2014["FCF"] - df_merged_x_2013["FCF"]) / np.abs(df_merged_x_2013["FCF"])
#df_merged_c_2014['총차입부채'] = (df_merged_z_2014['총차입부채'] - df_merged_x_2013['총차입부채']) / np.abs(df_merged_x_2013['총차입부채'])
df_merged_t_2014['자산총계'] = (df_merged_z_2014['자산총계'] - df_merged_x_2013['자산총계']) / np.abs(df_merged_x_2013['자산총계'])
#df_merged_c_2014['EBIT_매출액'] = (df_merged_z_2014['EBIT_매출액'] - df_merged_x_2013['EBIT_매출액']) / np.abs(df_merged_x_2013['EBIT_매출액'])
df_merged_t_2014['부채비율'] = (df_merged_z_2014['부채비율'] - df_merged_x_2013['부채비율']) / np.abs(df_merged_x_2013['부채비율'])
#df_merged_c_2014['차입금의존도'] = (df_merged_z_2014['차입금의존도'] - df_merged_x_2013['차입금의존도']) / np.abs(df_merged_x_2013['차입금의존도'])
#df_merged_c_2014['Marcap'] = (df_merged_z_2014['Marcap'] - df_merged_x_2013['Marcap']) / np.abs(df_merged_x_2013['Marcap'])
df_merged_t_2014['ESG등급'] = (df_merged_z_2014['ESG등급'] - df_merged_x_2013['ESG등급'])
df_merged_t_2014['지배구조'] = (df_merged_z_2014['지배구조'] - df_merged_x_2013['지배구조'])
df_merged_t_2014['사회'] = (df_merged_z_2014['사회'] - df_merged_x_2013['사회'])
df_merged_t_2014['환경'] = (df_merged_z_2014['환경'] - df_merged_x_2013['환경'])

In [ ]:
df_merged_t_2014.describe()

In [ ]:
df_merged_c_2014.describe()